In [1]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from sklearn.model_selection import train_test_split
from PIL import Image


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
# Load dataset from CSV file
data = pd.read_csv("/content/drive/MyDrive/data.csv")
print(data.head())  # Display first few rows


           Image  label
0  BikesHelmets0      1
1  BikesHelmets1      1
2  BikesHelmets2      1
3  BikesHelmets3      1
4  BikesHelmets4      1


In [ ]:
# Load images into a list
x = []
for i in data['Image']:
    img_path = f'/content/drive/MyDrive/helmet/file/{i}.png'
    if os.path.exists(img_path):
        img = Image.open(img_path)
        img = img.resize((64,64)).convert('RGB')
        x.append(np.array(img))
    else:
        print(f"❌ File not found: {img_path}")

# Convert list to numpy array
x = np.array(x)
y = np.array(data['label'][:len(x)])  # Ensure labels match the number of images

# Normalize images
x = x / 255.0

print(f"✅ Loaded {len(x)} images.")


❌ File not found: /content/drive/MyDrive/helmet/file/BikesHelmets50.png
✅ Loaded 749 images.


In [ ]:
# Split dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

print(f"Training Samples: {len(x_train)}, Testing Samples: {len(x_test)}")


Training Samples: 599, Testing Samples: 150


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

print("✅ Model created successfully!")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ Model created successfully!


In [ ]:
history = model.fit(x_train, y_train, validation_split=0.1, epochs=10, batch_size=32)

print("✅ Training Complete!")


Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 317ms/step - accuracy: 0.6257 - loss: 0.6991 - val_accuracy: 0.7000 - val_loss: 0.6124
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 174ms/step - accuracy: 0.6935 - loss: 0.6277 - val_accuracy: 0.7000 - val_loss: 0.6555
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 325ms/step - accuracy: 0.7086 - loss: 0.6441 - val_accuracy: 0.7000 - val_loss: 0.6197
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 174ms/step - accuracy: 0.7024 - loss: 0.6218 - val_accuracy: 0.7000 - val_loss: 0.6099
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 219ms/step - accuracy: 0.6996 - loss: 0.6086 - val_accuracy: 0.7000 - val_loss: 0.6113
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 176ms/step - accuracy: 0.7188 - loss: 0.5997 - val_accuracy: 0.7000 - val_loss: 0.6092
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 6s 232ms/step - accuracy: 0.7212 - loss: 0.5772 - val_accuracy: 0.7000 - val_loss: 0.6299
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 176ms/step - accuracy: 0.6970 - loss: 0.6026 - val_accuracy: 0.

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"🧪 Test Accuracy: {accuracy:.2f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6108 - loss: 0.6651
🧪 Test Accuracy: 0.63


In [ ]:
model.save("/content/helmet_model.h5")
print("✅ Model saved as helmet_model.h5")


✅ Model saved as helmet_model.h5


In [ ]:
# Convert to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
tflite_model_path = "/content/helmet_model.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"✅ Model converted and saved as {tflite_model_path}")


Saved artifact at '/tmp/tmp3aq5qag3'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  134158338542480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134158338543248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134158338544016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134158338543824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134158338544208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134158338543440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134158338545168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134158338545552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134158338545936: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134158338547472: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Model converted 

In [ ]:
from google.colab import files
files.download(tflite_model_path)  # Download normal TFLite model



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>